In [1]:
import environment as env
import random # Policies
from IPython.display import clear_output # Used in testing to clear cell output

In [6]:
# Game Play Policies for "AI" players

def policy_program(actions):
    """
    Programmatic policy - illustrate game play using pure programming logic (no machine learning) with simple conditions
    
    Policy will select in cascading order from the below:
        Play highest Distance card
        Play random Remedy card
        Play random non-discard option
        Discard a random card
    
    In addition, the policy will:
        Always choose to play a Coup Fourre when presented with the opportunity
        Never choose Extension play
    
    Arguments:
        actions ([int]) - actions the player can select from
    
        hand (list) - current player's hand, a list of Card objects: [Card]
    
    Return:
        int - the action to take
    """
    
    # Coup Fourre Option
    if 80 in actions or 85 in actions or 90 in actions:
        # Coup Fourre option - always play the coup fourre option (there will be only two options)
        return min(actions)
    
    # Extension Option
    if actions[0] > 90:
        # Extension option - always select "No"
        return max(actions)
    
    # Distance cards
    options = set(actions).intersection(list(range(19,24)) + list(range(38,43)) + list(range(57,62)))
    if len(options) > 0:
        return max(options)
    
    # Remedy cards
    options = set(actions).intersection(list(range(24,29)) + list(range(43,48)) + list(range(62,67)))
    if len(options) > 0:
        return random.choice(list(options))
    
    # Non-discard option
    options = set(actions).intersection(list(range(19,76)))
    if len(options) > 0:
        return random.choice(list(options))
    
    # Discard random card
    return random.choice(actions)
    

In [3]:
# Helper methods to simplify game play code

def print_action(label, actions):
    print("   {0}:".format(label))
    for act_idx in actions:
        act = game.action_matrix[act_idx]
        print("      {0} = {1}: {2}".format(act_idx, act[1], act[2]))

In [ ]:
# Player Names - enter player names in this list (2, 3, 4, or 6 players), give names to "AI" players as well
players = ['Dad', 'Bob']

# Policies - enter policy for each player (same number of policies as players above), options:
#     human - player will be prompted to on their turn for the action they wish to take
#     program - programmatic policy - simple policy that does not use machine learning
policies = ['human', 'program']

# Start the game
game = env.Game(players)

while game.play_status < 4:
    player_policy = policies[players.index(game.player_current.name)]
    
    if player_policy == 'human':
        # Human player
        
        # Menu Option
        menu_option = ""
        
        while menu_option not in ['n','h','t','q']:
            clear_output()
            print("Enter menu option:")
            print(f"  n = Load Next Player ({game.player_current.name})")
            print("  h = Display play history")
            print("  t = Display all team details")
            print("  q = Quit game")
            menu_option = input()
        
        if menu_option == "n":
            # Load player info
            clear_output()
            print(f"Current Player: {game.player_current.name}")
            print("Cards left in deck: {0}".format(len(game.deck.cards)))

            print("")
            for t in game.teams:
                print(f"---- {t.name}")
                print("No speed limit" if t.speed_status == 0 else "Speed Limit")
                print(["Out of Gas", "Flat Tire", "Accident", "Stop", "Go"][t.battle_status])
                print(f"Distance Traveled: {t.distance_points} ({t.distance_200} 200's played)")
                print("Safeties: {0}".format(', '.join(t.safety_played)))
                print("")

            print("Press enter to show hand and possible actions")
            show_hand = input()

            print("Hand:")
            for c in game.player_current.hand:
                print("   {0}: {1}".format(c.type, c.value))

            print("")
            print("Actions:")
            act_team_1 = [act for act in game.player_actions if game.action_matrix[act][0] == 0]
            act_team_2 = [act for act in game.player_actions if game.action_matrix[act][0] == 1]
            act_team_3 = [act for act in game.player_actions if game.action_matrix[act][0] == 2]
            act_discard = [act for act in game.player_actions if game.action_matrix[act][0] == -1]

            if len(act_team_1) > 0: print_action(game.teams[0].name, act_team_1)
            if len(act_team_2) > 0: print_action(game.teams[1].name, act_team_2)
            if len(act_team_3) > 0: print_action(game.teams[2].name, act_team_3)
            if len(act_discard) > 0: print_action("Discard", act_discard)

            print("")
            action = -1
            while isinstance(action, int) == False or action < 0 or action > 96:
                print("Enter Action number to play:")
                try:
                    action = int(input())
                except:
                    action = -1

            game.play_action(action)

        elif menu_option == 'h':
            # Game play history
            clear_output()
            print("-------- Action History ----------")
            for act_log in game.action_history:
                act = game.action_matrix[act_log[2]]
                print("{0} - {1} - {2}: {3} - Reward: {4}".format(act_log[0].name, game.teams[act[0]].name if act[0] > -1 else "Discard", act[1], act[2], act_log[3]))

            print("Press enter to continue")
            input()
        
        elif menu_option == 't':
            # Team details
            clear_output()
            for t in game.teams:
                print(f"------- {t.name} ---------")
                print("Speed Status: {0} ({1})".format("No speed limit" if t.speed_status == 0 else "Speed Limit", t.speed_status))
                print(["Out of Gas", "Flat Tire", "Accident", "Stop", "Go"][t.battle_status])
                print(f"Distance Points: {t.distance_points}")
                print(f"Distance 200: {t.distance_200}")
                print("Safeties: {0}".format(', '.join(t.safety_played)))

                print("")

                print("Speed Pile")
                for c in t.speed_pile:
                    print("  {0}: {1}".format(c.type, c.value))

                print("")

                print("Battle Pile")
                for c in t.battle_pile:
                    print("  {0}: {1}".format(c.type, c.value))

                print("")

                print("Distance Pile")
                for c in t.distance_pile:
                    print("  {0}: {1}".format(c.type, c.value))

                print("")

                print("Safety Pile")
                for c in t.safety_pile:
                    print("  {0}: {1}".format(c.type, c.value))

                print("")

            print("Press enter to continue")
            input()
            
        elif menu_option == 'q':
            # Quit Game
            game.play_status = 4
            
        
    else:
        # Program policy
        action = policy_program(game.player_actions)
        game.play_action(action)
        

# Game over
clear_output()
print("Game Over")
team_points = game.final_team_points()
for i in range(len(team_points)):
    print(f"{game.teams[i].name}: {team_points[i]} points")
